In [7]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import image_to_train as tr
from model import get_model
import test_func as test

# Test Model on the Urban100 Dataset

In [8]:
# Run this script when preprocessing Urban100 dataset
ds_path = os.path.join('Urban 100', 'X2 Urban100', 'X2')
HR_path = os.path.join(ds_path, 'HIGH X2 Urban')
LR_path = os.path.join(ds_path, 'LOW X2 Urban')

files_Y = sorted([os.path.join(HR_path, filename) for filename in os.listdir(HR_path)])
files_X = sorted([os.path.join(LR_path, filename) for filename in os.listdir(LR_path)])

files_Y = tf.convert_to_tensor(files_Y, tf.string)
files_X = tf.convert_to_tensor(files_X, tf.string)

def map_func(file):
    image = tf.io.read_file(file)
    image = tf.image.decode_image(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tr.to_luminance(image)                          # remove if you are going to compare the colored images
    return tf.squeeze(image)

Y = [tf.expand_dims(map_func(im), axis=-1) for im in files_Y]
X_sub = [tf.image.decode_image(tf.io.read_file(x), dtype=tf.float32) for x in files_X]
X_sub = [tr.preprocess_single_image(x, channel_last=True, batch_dimension=True) for x in X_sub]

In [9]:
# load self-trained weights
test_model = get_model()
test_model.load_weights(os.path.join('saved_weights', 'exp', '240'))

In [10]:
pred_ims = []
ssim_scores = []
psnr_scores = []

for x in X_sub:
    y_pred = test_model(x)
    pred_ims.append(tf.expand_dims(tr.bands_to_image(x+y_pred), axis=-1))
    
for x, y in zip(pred_ims, Y):
    ssim_scores.append(tf.image.ssim(x, y, max_val=1.0))
    psnr_scores.append(tf.image.psnr(x, y, max_val=1.0))

avg_ssim_score = sum(ssim_scores) / len(ssim_scores)
avg_psnr_score = sum(psnr_scores) / len(psnr_scores)
print('psnr score for trained model: {} \nssim score for trained model: {}'.format(avg_psnr_score, avg_ssim_score))

psnr score for trained model: 27.240066528320312 
ssim score for trained model: 0.8813284039497375


# Comparing with the Base_x2 Model (Model Used in Paper)

In [11]:
# load weights trained by the author
x2_weight_path = os.path.join('saved_weights', 'Weightx2', 'x2.ckpt')
model = get_model()
x2_model = test.load_x2_from_weights(model, x2_weight_path)

In [12]:
pred_ims_x2 = []
ssim_scores_x2 = []
psnr_scores_x2 = []
for x in X_sub:
    y_pred = x2_model(x)
    pred_ims_x2.append(tf.expand_dims(tr.bands_to_image(x+y_pred), axis=-1))

for x, y in zip(pred_ims_x2, Y):
    ssim_scores_x2.append(tf.image.ssim(x, y, max_val=1.0))
    psnr_scores_x2.append(tf.image.psnr(x, y, max_val=1.0))
avg_ssim_score_x2 = sum(ssim_scores_x2) / len(ssim_scores_x2)
avg_psnr_score_x2 = sum(psnr_scores_x2) / len(psnr_scores_x2)

print('psnr score for trained model: {} \nssim score for trained model: {}'.format(avg_psnr_score, avg_ssim_score))
print('psnr score for base x2: {} \nssim score for base x2: {}'.format(avg_psnr_score_x2, avg_ssim_score_x2))

# believe that the Urban100 scores is different due to changes in the original dataset since the original paper produced higher scores for PSNR and SSIM
# Model trained by self have lower scored since they were trained on bicubic downsampled images rather than the LR dataset

psnr score for trained model: 27.240066528320312 
ssim score for trained model: 0.8813284039497375
psnr score for base x2: 28.896934509277344 
ssim score for base x2: 0.9075761437416077


# Testing Model with Bicubic Downsampling on Arbitrary Datasets 
# (Not Accurate)

In [35]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import image_to_train as tr
from model import get_model
import test_func as test

downsample_scale = 0.5

In [45]:
HR_path = os.path.join('Urban 100', 'X2 Urban100', 'X2', 'HIGH X2 Urban')
files = [os.path.join(HR_path, filename) for filename in sorted(os.listdir(HR_path))[:20]]
# files = ['IMG_0275 2.png']

In [46]:
im_Y = [tf.expand_dims(test.upsample(p, scale=1), axis=-1) for p in files]
downsample_images = [test.downsample(p, downsample_scale, batch=False) for p in files]
downsample_sub_bands = [tf.expand_dims(tr.dwt_transform(downsample_image, channel_last=True), axis=0) for downsample_image in downsample_images]

In [47]:
model_test = get_model()
x2_model = get_model()
x2_weight_path = os.path.join('saved_weights', 'Weightx2', 'x2.ckpt')

x2_model = test.load_x2_from_weights(x2_model, x2_weight_path)
model_test.load_weights(os.path.join('saved_weights', 'exp', '240'))

In [49]:
model_test_pred_ims = []
x2_pred_ims = []

for x in downsample_sub_bands:
    model_test_y_pred = model_test(x)
    model_test_pred_ims.append(tf.expand_dims(tr.bands_to_image(x+model_test_y_pred), axis=-1))

    x2_y_pred = x2_model(x)
    x2_pred_ims.append(tf.expand_dims(tr.bands_to_image(x+x2_y_pred), axis=-1))

In [51]:
x2_psnr_score = test.calculate_psnr(x2_pred_ims, im_Y)
model_test_psnr_score = test.calculate_psnr(model_test_pred_ims, im_Y)

x2_ssim_score = test.calculate_ssim(x2_pred_ims, im_Y)
model_test_ssim_score = test.calculate_ssim(model_test_pred_ims, im_Y)

downsample_images_channel = [tf.expand_dims(downsample_image, axis=-1) for downsample_image in downsample_images]
baseline_score_ssim = test.calculate_ssim(downsample_images_channel, im_Y)
baseline_score_psnr = test.calculate_psnr(downsample_images_channel, im_Y)

print('psnr score for trained model: {} \nssim score for trained model: {}'.format(
    model_test_psnr_score, model_test_ssim_score))
print('psnr score for x2 model: {} \nssim score for x2 model: {}'.format(x2_psnr_score, x2_ssim_score))
print('psnr score for bicubic: {} \nssim score for bicubic: {}'.format(baseline_score_psnr, baseline_score_ssim))

psnr score for trained model: 28.235187530517578 
ssim score for trained model: 0.9148169755935669
psnr score for x2 model: 23.624568939208984 
ssim score for x2 model: 0.8414012789726257
psnr score for bicubic: 25.59308624267578 
ssim score for bicubic: 0.8643248677253723


In [ ]:

test.write_images_to_path('saved_images', tf.image.encode_png())
test.write_images_to_path('saved_images', )
test.write_images_to_path('saved_images', )